In [1]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy

bRedownloadLatest = False # will download feature class from agol, otherwise will use last download
bExportFeatureClass = True # export feature class
bClearFlags = True # will clear all flag fields from backup
bClearOverrides = False # will clear all override fields from backup

In [2]:
# Prep flag categories

# flag name must be short enough to be in dbf column name
dfFlags = pd.DataFrame([
    ['FL_REV'   , 'Not reviewed'                          , "([NOTES]=='') & ([NOTES_FURREV]=='')"],
    ['FL_LTPRV' , 'Less than previous forecast year'      , "(([MF2023]+[ADJ2023])<([M2019]+[aadtAdjFactor])) | (([MF2028]+[ADJ2028])<([MF2023]+[ADJ2023])) | (([MF2032]+[ADJ2032])<([MF2028]+[ADJ2028])) | (([MF2042]+[ADJ2042])<([MF2032]+[ADJ2032])) | (([MF2050]+[ADJ2050])<([MF2042]+[ADJ2042]))"],
    ['FL_ZERO'  , 'Zero volume'                           , "(([MF2023]+[ADJ2023])<=0) | (([MF2028]+[ADJ2028])<=0) | (([MF2032]+[ADJ2032])<=0) | (([MF2042]+[ADJ2042])<=0) | (([MF2050]+[ADJ2050])<=0)"],
    ['FL_HIADJ' , 'Model Adjustment Factor >95% base year', "((([aadtAdjFactor]/([M2019]+[aadtAdjFactor]))>0.95) | (([aadtAdjFactor]/([M2019]+[aadtAdjFactor]))<-0.95))"],
    ['FL_50LT19', 'HPMS: 2050 less than 2019'             , "(([MF2050]+[ADJ2050])<([M2019]+[aadtAdjFactor]))"],
    ['FL_503X19', 'HPMS: 2050 more than three times 2019' , "(([MF2050]+[ADJ2050])>(3*([M2019]+[aadtAdjFactor])))"],
    ['FL_FURREV', 'Further Review'                        , "([NOTES_FURREV]!='')"]
], columns=('flagName','flagDescription','flagCriteria'))

dfFlags.to_json('_site/data/flags.json', orient='records')

In [3]:
# CO_FIPS and SUBAREAID
forecastarea_mapping = {
    ( 0, 0): {'CO_NAME': ''          , 'F_AREA':'No County'      },
    ( 0, 1): {'CO_NAME': ''          , 'F_AREA':'No County'      },
    ( 0, 2): {'CO_NAME': ''          , 'F_AREA':'No County'      },
    ( 0, 3): {'CO_NAME': ''          , 'F_AREA':'No County'      },
    ( 0, 4): {'CO_NAME': ''          , 'F_AREA':'No County'      },
    ( 0, 5): {'CO_NAME': ''          , 'F_AREA':'No County'      },

    ( 1, 0): {'CO_NAME': 'Beaver'    , 'F_AREA':'UDOT'           },
    ( 3, 0): {'CO_NAME': 'Box Elder' , 'F_AREA':'UDOT'           },
    ( 3, 1): {'CO_NAME': 'Box Elder' , 'F_AREA':'WFRC'           },
    ( 5, 0): {'CO_NAME': 'Cache'     , 'F_AREA':'UDOT'           },
    ( 5, 2): {'CO_NAME': 'Cache'     , 'F_AREA':'Cache'          },
    ( 7, 0): {'CO_NAME': 'Carbon'    , 'F_AREA':'UDOT'           },
    ( 9, 0): {'CO_NAME': 'Daggett'   , 'F_AREA':'UDOT'           },
    (11, 1): {'CO_NAME': 'Davis'     , 'F_AREA':'WFRC'           },
    (13, 0): {'CO_NAME': 'Duchesne'  , 'F_AREA':'UDOT'           },
    (15, 0): {'CO_NAME': 'Emery'     , 'F_AREA':'UDOT'           },
    (17, 0): {'CO_NAME': 'Garfield'  , 'F_AREA':'UDOT'           },
    (19, 0): {'CO_NAME': 'Grand'     , 'F_AREA':'UDOT'           },
    (21, 5): {'CO_NAME': 'Iron'      , 'F_AREA':'Iron'           },
    (23, 0): {'CO_NAME': 'Juab'      , 'F_AREA':'UDOT'           },
    (25, 0): {'CO_NAME': 'Kane'      , 'F_AREA':'UDOT'           },
    (27, 0): {'CO_NAME': 'Millard'   , 'F_AREA':'UDOT'           },
    (29, 0): {'CO_NAME': 'Morgan'    , 'F_AREA':'UDOT'           },
    (31, 0): {'CO_NAME': 'Piute'     , 'F_AREA':'UDOT'           },
    (33, 0): {'CO_NAME': 'Rich'      , 'F_AREA':'UDOT'           },
    (35, 1): {'CO_NAME': 'Salt Lake' , 'F_AREA':'WFRC'           },
    (37, 0): {'CO_NAME': 'San Juan'  , 'F_AREA':'UDOT'           },
    (39, 0): {'CO_NAME': 'Sanpete'   , 'F_AREA':'UDOT'           },
    (41, 0): {'CO_NAME': 'Sevier'    , 'F_AREA':'UDOT'           },
    (43, 4): {'CO_NAME': 'Summit'    , 'F_AREA':'Summit'         },
    (45, 0): {'CO_NAME': 'Tooele'    , 'F_AREA':'UDOT'           },
    (47, 0): {'CO_NAME': 'Uintah'    , 'F_AREA':'UDOT'           },
    (49, 1): {'CO_NAME': 'Utah'      , 'F_AREA':'MAG'            },
    (51, 4): {'CO_NAME': 'Wasatch'   , 'F_AREA':'MAG'            },
    (53, 3): {'CO_NAME': 'Washington', 'F_AREA':'Dixie'          },
    (55, 0): {'CO_NAME': 'Wayne'     , 'F_AREA':'UDOT'           },
    (57, 0): {'CO_NAME': 'Weber'     , 'F_AREA':'UDOT'           },
    (57, 1): {'CO_NAME': 'Weber'     , 'F_AREA':'WFRC'           }
}


# Create JSONs

In [4]:
# convert csvs to jsons
lstSegidFiles = ['aadt', 'linear-forecasts', 'model-forecasts', 'previous-forecasts']
lstNonSegidFiles = ['aadt-sources', 'previous-forecasts-sources', 'projection-groups']

for file in lstSegidFiles + lstNonSegidFiles:
    # Read CSV file
    pd.read_csv('intermediate/' + file + '.csv').to_json('_site/data/' + file + '.json', orient='records')

# Create Segment Feature Class

In [5]:
fnSegments = 'data/segments/Final_Segments_BMP_EMP.shp'

seg_cols = ['SEGID','CO_FIPS','PLANAREA','SUBAREAID','SHAPE'] ############## ADD SUBAREA ID WHEN FILE HAS IT #####################

# USE ALL PLANAREAS since this app will be for entire state
# filter by PLANAREA in segments shapefile
#filterPlanArea = ['WFRC','MAG'] # must be an array... if only single item, the still include []

In [6]:
if bRedownloadLatest:

    # Define the URL
    url = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Forecasts_gdb/FeatureServer/0/query"

    # Base parameters for the request
    params = {
        "f": "json",
        "where": "1=1",
        "outFields": "*",
        "returnGeometry": "false",  # Assuming you only want attributes
    }

    all_features = []

    # Pagination parameters
    batch_size = 2000
    offset = 0

    while True:
        # Adjust parameters for pagination
        params['resultOffset'] = offset
        params['resultRecordCount'] = batch_size

        # Send the request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])
            
            # Check if no features were returned, which means we're done
            if not features:
                break
            
            # Append features to the master list
            all_features.extend(features)
            
            # Increase the offset for next iteration
            offset += batch_size

        else:
            print("Error fetching data:", response.status_code)
            break

    # Extract attribute table
    attribute_table = [feature['attributes'] for feature in all_features]

    # Convert to DataFrame
    df = pd.DataFrame(attribute_table)

    from datetime import datetime

    # Generate a filename with the current timestamp in the specified folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"backup\\forecastsegments_backup_{timestamp}.csv"

    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
else:
    print("didn't download... will use latest")

didn't download... will use latest


In [7]:
# Define the URL
url = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/forecasts_logfile/FeatureServer/0/query"

# Base parameters for the request
params = {
    "f": "json",
    "where": "1=1",
    "outFields": "*",
    "returnGeometry": "false",  # If it's a stand-alone table, geometry isn't required.
    "outSR": "4326",  # Specify output spatial reference if needed
    "returnExceededLimitFeatures": "true"
}

all_features = []

# Pagination parameters
batch_size = 2000
offset = 0

while True:
    # Adjust parameters for pagination
    params['resultOffset'] = offset
    params['resultRecordCount'] = batch_size

    # Send the request
    response = requests.post(url, params=params)  # Use POST since some servers may have GET length limits

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        features = data.get('features', [])
        
        # Check if no features were returned, which means we're done
        if not features:
            break
        
        # Append features to the master list
        all_features.extend(features)
        
        # Increase the offset for next iteration
        offset += batch_size

    else:
        print("Error fetching data:", response.status_code)
        break

# Extract attribute table
attribute_table = [feature['attributes'] for feature in all_features]

# Convert to DataFrame
df = pd.DataFrame(attribute_table)

from datetime import datetime

# Generate a filename with the current timestamp in the specified folder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"backup\\logfile_backup_{timestamp}.csv"

# Save the DataFrame to a CSV file
df.to_csv(filename, index=False)
print(f"Data saved to {filename}")

Data saved to backup\logfile_backup_20230925_162541.csv


In [8]:
#import datetime
#
## empty log file for AGOL
#dfLogFile = pd.DataFrame([
#    ['dummy','dummy',0,0,0,0,0,'dummy',datetime.datetime.now()]
#],columns=('SEGID','EDITKEY','ADJ2023','ADJ2028','ADJ2032','ADJ2042','ADJ2050','NOTES','TIMESTAMP'))
#
#dfLogFile.to_csv('results/forecasts-logfile.csv')
#
#dfLogFile

In [9]:
# Get data from last backup

import os

# List all backup files in the directory
backup_dir = "backup\\"
all_files = [f for f in os.listdir(backup_dir) if f.startswith('forecastsegments_backup_') and f.endswith('.csv')]

# Sort the files based on their timestamps
sorted_files = sorted(all_files, reverse=True)  # Latest timestamp will be first

# Check if there are any backup files
if sorted_files:
    latest_file = os.path.join(backup_dir, sorted_files[0])
    # Load the most recent backup into a dataframe
    dfLatest = pd.read_csv(latest_file,low_memory=False)

    # when duplicate errors we need to get rid of extra data
    dfLatest = dfLatest.groupby('SEGID').first().reset_index()

    print(f"Loaded {latest_file} into a DataFrame")

    if bClearFlags:
        # Specify columns to delete
        columns_to_delete = [col for col in dfLatest.columns if col.startswith('FL_')]

        # Remove the specified columns from df_filtered
        dfLatest.drop(columns_to_delete, axis=1, inplace=True)
    
        print(f"Dropped FL_ columns")

    if bClearOverrides:
        # Specify columns to delete
        columns_to_delete = [col for col in dfLatest.columns if col.startswith('OV_')]

        # Remove the specified columns from df_filtered
        dfLatest.drop(columns_to_delete, axis=1, inplace=True)
    
        print(f"Dropped OV_ columns")
else:
    dfLatest = pd.DataFrame()
    print("No backup files found in the directory.")

# You can then use dfLatest as your DataFrame


Loaded backup\forecastsegments_backup_20230922_130440.csv into a DataFrame
Dropped FL_ columns


In [10]:
# read in segments, filter, and select only key columns
sdfSegments = pd.DataFrame.spatial.from_featureclass(fnSegments)
#sdfSegments = sdfSegments[sdfSegments['PLANAREA'].isin(filterPlanArea)]
sdfSegments = sdfSegments[seg_cols]

display(sdfSegments)

,SEGID,CO_FIPS,PLANAREA,SUBAREAID,SHAPE
0,0006_000.0,27,UDOT,0,"{""paths"": [[[236177.7000000002, 4327541.25], [..."
1,0006_000.7,27,UDOT,0,"{""paths"": [[[237241.1799999997, 4327399.720000..."
2,0006_016.0,27,UDOT,0,"{""paths"": [[[261403.90000000037, 4327045.4], [..."
3,0006_046.0,27,UDOT,0,"{""paths"": [[[305380.16000000015, 4325741.85], ..."
4,0006_060.2,27,UDOT,0,"{""paths"": [[[324618, 4337936.9], [324647.40000..."
...,...,...,...,...,...
8717,0056_060.2,21,UDOT,5,"{""paths"": [[[316466.93000000063, 4172440.09999..."
8718,0056_060.6,21,UDOT,5,"{""paths"": [[[317122.6400000015, 4172430.75], [..."
8719,0056_060.5,21,UDOT,5,"{""paths"": [[[316687.7100000009, 4172438.000000..."
8720,0056_060.4,21,UDOT,5,"{""paths"": [[[316617.4350000005, 4172438.9275],..."


In [11]:
# read in intermediate data
dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')
dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'M' + dfModVolAdj['YEAR']

dfModVolAdj_pvModAadt = dfModVolAdj.pivot_table(index=['SEGID','SUBAREAID','FAC_WDAVG','aadtAdjFactor'], columns='YEAR', values='modAadt')
dfModVolAdj_pvModAadt.reset_index(inplace=True)

dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')

dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'MF' + dfModVolAdj['YEAR'].str.replace(".0","", regex=False)

dfModVolAdj_pvModAadtAdj = dfModVolAdj.pivot_table(index=['SEGID','SUBAREAID','FAC_WDAVG','aadtAdjFactor'], columns='YEAR', values='modForecast')
dfModVolAdj_pvModAadtAdj.reset_index(inplace=True)

dfModVolAdj = pd.DataFrame.merge(dfModVolAdj_pvModAadt, dfModVolAdj_pvModAadtAdj, on=('SEGID','SUBAREAID','FAC_WDAVG','aadtAdjFactor'))

#dfModVolAdj.columns = [str(col).split('.')[0] if '.' in str(col) else str(col) for col in dfModVolAdj.columns]


In [12]:
# join segment data and forecast data
_sdf = pd.DataFrame.merge(dfModVolAdj, sdfSegments, on=('SEGID','SUBAREAID'), how='right')

# add columns for Adjustments
_sdf['ADJ2019'] = 0
_sdf['ADJ2023'] = 0
_sdf['ADJ2028'] = 0
_sdf['ADJ2032'] = 0
_sdf['ADJ2042'] = 0
_sdf['ADJ2050'] = 0
_sdf['ADJHIST'] = 0
_sdf['NOTES'] = ""
_sdf['NOTES_FURREV'] = ""

# replace with latest file
if dfLatest.empty:
    print("The latest dataframe is empty.")
else:
    print("The latest dataframe is not empty.")
        
    # Specify columns to include from latest download (FLAGS WILL NOT BE INCLUDED WHEN FILTERED PRIOR TO THIS STEP)
    columns_to_include = [col for col in dfLatest.columns if col.startswith('ADJ') or col.startswith('FL_') or col.startswith('OV_') or col == 'SEGID' or col == 'NOTES' or col == 'NOTES_FURREV']

    # Filter df_filtered to only have the specified columns
    dfLatest_filtered = dfLatest[columns_to_include]

    # Merge sdfData with df_filtered on SEGID
    _sdfWithLatest = _sdf.merge(dfLatest_filtered, on='SEGID', how='left', suffixes=('_delete', ''))

    # Drop any duplicate columns (those with '_delete' suffix) after the merge 
    for column in _sdfWithLatest.columns:
        if column.endswith('_delete'):
            _sdfWithLatest.drop(column, axis=1, inplace=True)

    _sdfWithLatest['NOTES'].fillna('0', inplace=True)
    _sdfWithLatest['NOTES'] = _sdfWithLatest['NOTES'].astype(str)
    _sdfWithLatest.loc[(_sdfWithLatest['NOTES']=='0'), 'NOTES'] = ''
    _sdfWithLatest['NOTES'] = _sdfWithLatest['NOTES'].str.strip()

    _sdfWithLatest['NOTES_FURREV'].fillna('0', inplace=True)
    _sdfWithLatest['NOTES_FURREV'] = _sdfWithLatest['NOTES_FURREV'].astype(str)
    _sdfWithLatest.loc[(_sdfWithLatest['NOTES_FURREV']=='0'), 'NOTES_FURREV'] = ''
    _sdfWithLatest['NOTES_FURREV'] = _sdfWithLatest['NOTES_FURREV'].str.strip()

# add flag columns
# Loop through the rows of 'dfFlags'
for index, row in dfFlags.iterrows():
    # Get the flag name from the 'flagName' column
    flag_name = row['flagName']

    # Get the flag criteria from the 'flagCriteria' column (if needed)
    flag_criteria = row['flagCriteria']

    # Add a new column to 'dfSegs' with the flag name and set it equal to evaluated criteria, prepare criteria string with replace
    expression = "_sdfWithLatest['" + flag_name + "'] = " + flag_criteria.replace("[", "_sdfWithLatest['").replace("]", "']")
    print(expression)

    # execute expression!
    exec(expression)
    #print(result)  # Output will be 11
    
    if bClearOverrides:
        display('flags overwritten')
        # add override column
        _sdfWithLatest[flag_name.replace('FL_','OV_')] = 0

_sdfWithLatest.fillna(0,inplace=True)

sdfSegmentsWData = _sdfWithLatest
sdfSegmentsWData

The latest dataframe is not empty.
_sdfWithLatest['FL_REV'] = (_sdfWithLatest['NOTES']=='') & (_sdfWithLatest['NOTES_FURREV']=='')
_sdfWithLatest['FL_LTPRV'] = ((_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])<(_sdfWithLatest['M2019']+_sdfWithLatest['aadtAdjFactor'])) | ((_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])<(_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])) | ((_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])<(_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])) | ((_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042'])<(_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])) | ((_sdfWithLatest['MF2050']+_sdfWithLatest['ADJ2050'])<(_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042']))
_sdfWithLatest['FL_ZERO'] = ((_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])<=0) | ((_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])<=0) | ((_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])<=0) | ((_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042'])<=0) | ((_sdfWithLatest['M

,SEGID,SUBAREAID,FAC_WDAVG,aadtAdjFactor,M2019,M2023,M2028,M2032,M2042,M2050,...,OV_FURREV,NOTES,NOTES_FURREV,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV
0,0006_000.0,0,0.9840,289.0,126.0,99.0,101.0,103.0,107.0,111.0,...,0.0,,,True,True,False,False,True,False,False
1,0006_000.7,0,0.9840,202.0,213.0,216.0,215.0,214.0,214.0,216.0,...,0.0,,,True,True,False,False,True,False,False
2,0006_016.0,0,0.9840,339.0,76.0,73.0,70.0,67.0,65.0,62.0,...,0.0,,,True,True,False,False,True,False,False
3,0006_046.0,0,0.9840,296.0,76.0,73.0,70.0,67.0,65.0,62.0,...,0.0,,,True,True,False,False,True,False,False
4,0006_060.2,0,0.9840,292.0,80.0,76.0,73.0,70.0,68.0,65.0,...,0.0,,,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8717,0056_060.2,5,1.0924,-4564.0,17806.0,22593.0,26416.0,28384.0,34363.0,35340.0,...,0.0,,,True,False,False,False,False,False,False
8718,0056_060.6,5,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,,,True,False,False,False,False,False,False
8719,0056_060.5,5,1.0946,-2000.0,18915.0,24611.0,29116.0,31996.0,38428.0,39502.0,...,0.0,,,True,False,False,False,False,False,False
8720,0056_060.4,5,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,,,True,False,False,False,False,False,False


In [13]:
sdfSegmentsWData[['SEGID','NOTES_FURREV','FL_FURREV','OV_FURREV']]

,SEGID,NOTES_FURREV,FL_FURREV,OV_FURREV
0,0006_000.0,,False,0.0
1,0006_000.7,,False,0.0
2,0006_016.0,,False,0.0
3,0006_046.0,,False,0.0
4,0006_060.2,,False,0.0
...,...,...,...,...
8717,0056_060.2,,False,0.0
8718,0056_060.6,,False,0.0
8719,0056_060.5,,False,0.0
8720,0056_060.4,,False,0.0


In [14]:
manualoverride = False

if manualoverride: 
# fix manually:
    sdfSegmentsWData.loc[sdfSegmentsWData['SEGID']=='Dixie_5134','SUBAREAID']=3

    sdfSegmentsWData.loc[sdfSegmentsWData['SEGID']=='1822_000.0','CO_FIPS']=49

In [15]:
# Add CO_NAME and ForecastArea columns to the DataFrame using multi-index mapping
sdfSegmentsWData['CO_NAME'] = sdfSegmentsWData.apply(lambda row: forecastarea_mapping.get((row['CO_FIPS'], row['SUBAREAID']), {}).get('CO_NAME'), axis=1)
sdfSegmentsWData['F_AREA'] = sdfSegmentsWData.apply(lambda row: forecastarea_mapping.get((row['CO_FIPS'], row['SUBAREAID']), {}).get('F_AREA'), axis=1)

In [16]:
# export
#if bShapefileExport: #sdfSegmentsWData.spatial.to_featureclass('results/ForecastSegments/ForecastSegments.shp',sanitize_columns=False)
from arcgis.features import GeoAccessor

if bExportFeatureClass:
    # Define the geodatabase path
    gdb_path        = r"results\Forecasts.gdb"
    gdb_path_backup = r"backup\Forecasts_backup.gdb"

    # Check if the geodatabase exists, if not, create it
    if not arcpy.Exists(gdb_path):
        arcpy.CreateFileGDB_management(r"results", "Forecasts.gdb")
        
    # Check if the geodatabase exists, if not, create it
    if not arcpy.Exists(gdb_path_backup):
        arcpy.CreateFileGDB_management(r"backup", "Forecasts_backup.gdb")

    # Export SDF to the geodatabase as a feature class
    feature_class_name = "ForecastSegments"

    # Generate a filename with the current timestamp in the specified folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    feature_class_name_with_timestamp = f"ForecastSegments_{timestamp}"

    # file to replace
    sdfSegmentsWData.spatial.to_featureclass(location=f"{gdb_path_backup}\\{feature_class_name_with_timestamp}", sanitize_columns=False)
    print(f"Feature class {feature_class_name_with_timestamp} created in {gdb_path_backup}")

    # backup file
    sdfSegmentsWData.spatial.to_featureclass(location=f"{gdb_path}\\{feature_class_name}", sanitize_columns=False)

    print(f"Feature class {feature_class_name} created in {gdb_path}")

else:
    print ('not exported')


Feature class ForecastSegments_20230925_162546 created in backup\Forecasts_backup.gdb
Feature class ForecastSegments created in results\Forecasts.gdb


In [17]:
sdfSegmentsWData[sdfSegmentsWData['SEGID']=='1438_000.9']

,SEGID,SUBAREAID,FAC_WDAVG,aadtAdjFactor,M2019,M2023,M2028,M2032,M2042,M2050,...,NOTES_FURREV,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,CO_NAME,F_AREA
4063,1438_000.9,1,1.0924,3080.0,5428.0,5723.0,6925.0,1221.0,1273.0,6461.0,...,DUPLICATE SEGID!!,False,True,False,False,False,False,True,Davis,WFRC


In [18]:
duplicated_segid = sdfSegmentsWData[sdfSegmentsWData.duplicated('SEGID', keep=False)]['SEGID'].unique()
duplicated_segid

array([], dtype=object)

In [19]:
sdfSegmentsWData[sdfSegmentsWData['F_AREA'].isna()][['SEGID','CO_FIPS','SUBAREAID','F_AREA']]


,SEGID,CO_FIPS,SUBAREAID,F_AREA


In [20]:
# export segment JSON
jsonSegment = sdfSegmentsWData[['SEGID','PLANAREA','SUBAREAID','CO_NAME','F_AREA']]

jsonSegment.to_json('_site/data/segments.json', orient='records')

In [21]:
jsonSegment.groupby(['F_AREA','CO_NAME','SUBAREAID'],as_index=False).agg(numSegs=('SEGID','count'))

,F_AREA,CO_NAME,SUBAREAID,numSegs
0,Cache,Cache,2,489
1,Dixie,Washington,3,613
2,Iron,Iron,5,248
3,MAG,Utah,1,1398
4,MAG,Wasatch,4,174
5,No County,,0,3
6,No County,,1,129
7,No County,,3,2
8,No County,,5,1
9,Summit,Summit,4,205
